# ID3 Classification Trees: Perfect Split with Information Gain - Lab

## Introduction

In this lab, we will simulate the example from the previous lesson in Python. You will write functions to calculate entropy and IG which will be used for calculating these uncertainty measures and deciding upon creating a split using information gain while growing an ID3 classification tree. You will also write a general function that can be used for other (larger) problems as well. So let's get on with it.

## Objectives

In this lab you will: 

- Write functions for calculating entropy and information gain measures  
- Use entropy and information gain to identify the attribute that results in the best split at each node


## Problem

You will use the same problem about deciding whether to go and play tennis on a given day, given the weather conditions. Here is the data from the previous lesson:

|  outlook | temp | humidity | windy | play |
|:--------:|:----:|:--------:|:-----:|:----:|
| overcast | cool |   high   |   Y   |  yes |
| overcast | mild |  normal  |   N   |  yes |
|   sunny  | cool |  normal  |   N   |  yes |
| overcast |  hot |   high   |   Y   |  no  |
|   sunny  |  hot |  normal  |   Y   |  yes |
|   rain   | mild |   high   |   N   |  no  |
|   rain   | cool |  normal  |   N   |  no  |
|   sunny  | mild |   high   |   N   |  yes |
|   sunny  | cool |  normal  |   Y   |  yes |
|   sunny  | mild |  normal  |   Y   |  yes |
| overcast | cool |   high   |   N   |  yes |
|   rain   | cool |   high   |   Y   |  no  |
|   sunny  |  hot |  normal  |   Y   |  no  |
|   sunny  | mild |   high   |   N   |  yes |


## Write a function `entropy(pi)` to calculate total entropy in a given discrete probability distribution `pi`

- The function should take in a probability distribution `pi` as a list of class distributions. This should be a list of two integers, representing how many items are in each class. For example: `[4, 4]` indicates that there are four items in each class, `[10, 0]` indicates that there are 10 items in one class and 0 in the other. 
- Calculate and return entropy according to the formula: $$Entropy(p) = -\sum (P_i . log_2(P_i))$$

## Entropy Calculation

In [96]:
import math

def entropy(pi):
    """
    Return the entropy of a probability distribution:
    
    H(p) = -sum(p(x) * log(p(x))) for all x
    """
    # Convert frequencies to probabilities
    total = sum(pi)
    probabilities = [p / total for p in pi]
    
    # Calculate entropy
    entropy = -sum(p * math.log2(p) for p in probabilities if p > 0)
    return entropy

# Test the function
print(entropy([1, 1]))  # Maximum Entropy e.g. a coin toss
print(entropy([0, 6]))  # No entropy, ignore the -ve with zero , it's there due to log function
print(entropy([2, 10]))  # A random mix of classes


1.0
-0.0
0.6500224216483541


## Write a function `IG(D,a)` to calculate the information gain 

- As input, the function should take in `D` as a class distribution array for target class, and `a` the class distribution of the attribute to be tested
- Using the `entropy()` function from above, calculate the information gain as:

$$gain(D,A) = Entropy(D) - \sum(\frac{|D_i|}{|D|}.Entropy(D_i))$$

where $D_{i}$ represents distribution of each class in `a`.


## Filtering the frequencies of classes

In [132]:
import pandas as pd

play = pd.read_csv("./Data.csv", index_col=0)

# Example DataFrame



# Apply strip() to all values in the DataFrame, regardless of their type
play = play.applymap(lambda x: x.strip())

# Remove all white spaces within the strings in the entire DataFrame
play = play.applymap(lambda x: x.replace(' ', ''))



play.head()
play.columns

# Get value counts of 'outlook' based on 'play' being 'yes' or 'no'
outlook_yes = play[play['play'] == 'yes']['windy'].value_counts()
outlook_no = play[play['play'] == 'no']['windy'].value_counts()

print("Outlook distribution when 'play' is 'yes':")
print(outlook_yes)

print("Outlook distribution when 'play' is 'no':")
print(outlook_no)

Outlook distribution when 'play' is 'yes':
windy
N    5
Y    4
Name: count, dtype: int64
Outlook distribution when 'play' is 'no':
windy
Y    3
N    2
Name: count, dtype: int64


In [118]:
# Strip white spaces from column names
play.columns = play.columns.str.strip()

# Remove white spaces within column names
play.columns = play.columns.str.replace(' ', '')
play.columns = play.columns.str.replace('  ', '')
play.head()

,outlook,temp,humidity,windy,play
Sno,,,,,
1,overcast,cool,high,Y,yes
2,overcast,mild,normal,N,yes
3,sunny,cool,normal,N,yes
4,overcast,hot,high,Y,no
5,sunny,hot,normal,Y,yes


## Computation of information gain

In [98]:
import math

# Define the Information Gain function
def IG(D, a):
    entropy_D = entropy(D) # Calling the entropy function
    weighted_entropy = 0
    total = sum([sum(subset) for subset in a])
    for subset in a:
        subset_entropy = entropy(subset)
        weighted_entropy += (sum(subset) / total) * subset_entropy
    return entropy_D - weighted_entropy

## First iteration - Decide the best split for the root node

- Create the class distribution `play` as a list showing frequencies of both classes from the dataset
- Similarly, create variables for four categorical feature attributes showing the class distribution for each class with respect to the target classes (yes and no)
- Pass the play distribution with each attribute to calculate the information gain

In [99]:
# Class distribution (play)
play_distribution = [9, 5]  # [yes, no]

# Outlook attribute distribution
outlook_distribution = [
    [3, 1],  # overcast
    [6, 1],  # sunny
    [0, 3]   # rain
]

# Temperature attribute distribution
temp_distribution = [
    [4, 2],  # cool
    [4, 1],  # mild
    [1, 2]   # hot
]

# Humidity attribute distribution
humidity_distribution = [
    [4, 3],  # high
    [5, 2]   # normal
]

# Windy attribute distribution
windy_distribution = [
    [4, 3],  # Y
    [5, 2]   # N
]

# Calculate Information Gain for each attribute
IG_outlook = IG(play_distribution, outlook_distribution)
IG_temp = IG(play_distribution, temp_distribution)
IG_humidity = IG(play_distribution, humidity_distribution)
IG_windy = IG(play_distribution, windy_distribution)

print(f"Information Gain for Outlook: {IG_outlook}")
print(f"Information Gain for Temperature: {IG_temp}")
print(f"Information Gain for Humidity: {IG_humidity}")
print(f"Information Gain for Windy: {IG_windy}")


Information Gain for Outlook: 0.4126558195340009
Information Gain for Temperature: 0.09212146003297272
Information Gain for Humidity: 0.016111606370189824
Information Gain for Windy: 0.016111606370189824


We see here that the outlook attribute gives us the highest value for information gain, hence we choose this for creating a split at the root node. So far, we've built the following decision tree:
<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/outlook.png'  width ="650"  >


## Second iteration

Since the first iteration determines what split we should make for the root node of our tree, it's pretty simple. Now, we move down to the second level and start finding the optimal split for each of the nodes on this level. The first branch (edge) of three above that leads to the "Sunny" outcome. Of the temperature, humidity and wind attributes, find which one provides the highest information gain.

Follow the same steps as above. Remember, we have 6 positive examples and 1 negative example in the "sunny" branch.

In [137]:
### Filtering Sunny

# Filter for rows where 'play' is 'yes' and 'outlook' is 'sunny', then get 'windy' value counts
outlook_yes = play[(play['play'] == 'yes') & (play['outlook'] == 'sunny')]['temp'].value_counts()

# Filter for rows where 'play' is 'no' and 'outlook' is 'sunny', then get 'windy' value counts
outlook_no = play[(play['play'] == 'no') & (play['outlook'] == 'sunny')]['temp'].value_counts()

print("Windy distribution when 'play' is 'yes' and 'outlook' is 'sunny':")
print(outlook_yes)

print("Windy distribution when 'play' is 'no' and 'outlook' is 'sunny':")
print(outlook_no)

Windy distribution when 'play' is 'yes' and 'outlook' is 'sunny':
temp
mild    3
cool    2
hot     1
Name: count, dtype: int64
Windy distribution when 'play' is 'no' and 'outlook' is 'sunny':
temp
hot    1
Name: count, dtype: int64


In [138]:
# Class distribution for "Sunny" branch
sunny_play_distribution = [6, 1]  # [yes, no]

# Temperature attribute distribution for "Sunny" branch
sunny_temp_distribution = [
    [2, 0],  # cool
    [3, 0],  # mild
    [1, 1]   # hot
]

# Humidity attribute distribution for "Sunny" branch
sunny_humidity_distribution = [
    [2, 0],  # high
    [4, 1]   # normal
]

# Windy attribute distribution for "Sunny" branch
sunny_windy_distribution = [
    [3, 1],  # Y
    [3, 0]   # N
]

# Calculate Information Gain for each attribute in "Sunny" branch
IG_sunny_temp = IG(sunny_play_distribution, sunny_temp_distribution)
IG_sunny_humidity = IG(sunny_play_distribution, sunny_humidity_distribution)
IG_sunny_windy = IG(sunny_play_distribution, sunny_windy_distribution)

print(f"Information Gain for Temperature in 'Sunny' branch: {IG_sunny_temp}")
print(f"Information Gain for Humidity in 'Sunny' branch: {IG_sunny_humidity}")
print(f"Information Gain for Windy in 'Sunny' branch: {IG_sunny_windy}")

Information Gain for Temperature in 'Sunny' branch: 0.3059584928680418
Information Gain for Humidity in 'Sunny' branch: 0.0760098536627829
Information Gain for Windy in 'Sunny' branch: 0.12808527889139443


We see that temperature gives us the highest information gain, so we'll use it to split our tree as shown below:

<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/temp.png'  width ="650"  >


Let's continue. 

## Third iteration

We'll now calculate splits for the 'temperature' node we just created for days where the weather is sunny. Temperature has three possible values: [Hot, Mild, Cool]. This means that for each of the possible temperatures, we'll need to calculate if splitting on windy or humidity gives us the greatest possible information gain.

Why are we doing this next instead of the rest of the splits on level 2? Because a decision tree is a greedy algorithm, meaning that the next choice is always the one that will give it the greatest information gain. In this case, evaluating the temperature on sunny days gives us the most information gain, so that's where we'll go next.

## All other iterations

What happens once we get down to a 'pure' split? Obviously, we stop splitting. Once that happens, we go back to the highest remaining uncalculated node and calculate the best possible split for that one. We then continue on with that branch, until we have exhausted all possible splits or we run into a split that gives us 'pure' leaves where all 'play=Yes' is on one side of the split, and all 'play=No' is on the other.

## Summary 

This lab should have helped you familiarize yourself with how decision trees work 'under the hood', and demystified how the algorithm actually 'learns' from data by: 

- Calculating entropy and information gain
- Figuring out the next split you should calculate ('greedy' approach) 